In [7]:
import os
# Change the directory to reflect the main repository
os.chdir("C:/Users/rsilva/Google Drive/projects/programming/automation")

import sistemas
import sei
import functions
import page
from page import *

% reload_ext autoreload
% autoreload 2

In [5]:
usr = "rsilva"
pwd = "Savorthemom3nts"
browser = functions.init_browser(webdriver.Ie(), usr, pwd)

In [4]:
CPF = ["21979841870", "20048329835", "04396876670", "32252462809", "32149146851", "28482712802", 
       "02900107563", "02900107563", "17349224869", "21688805818", "13305102837", "8205179840",
       "95243178187", "10209990805", "41473019834", "03615339851", "11439066833", ]

In [15]:
scpx = sistemas.Scpx(browser.driver, usr, pwd)

In [7]:
scpx.consulta(CPF)

In [5]:
scpx.incluir_serviço(CPF)

In [6]:
scpx.aprovar_movimento(CPF, "B")

In [8]:
scpx.licenciar_estacao(CPF)

In [9]:
scpx.consulta(CPF)

In [17]:
for cpf in CPF[8:]:
    try:
        
        scpx.imprimir_licenca(cpf)
    except UnexpectedAlertPresentException as e:
        pass
    
    input()